# Vortex Level 0 → Level 1 <br/>

https://overthewire.org/wargames/vortex/vortex0.html


In [24]:
import socket
import struct

HOST = "vortex.labs.overthewire.org"
PORT = 5842

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))

    result = 0
    for i in range(0, 4):
        data = s.recv(4)
        print(data)
        result += int.from_bytes(data, "little")
        # s.sendall(data)

    s.sendall(struct.pack("<I", result))
    data = s.recv(1024)
    print(data)

b'\xf7\xee2\n'
b'\xfaP\x15\x10'
b'\x870/-'
b'\xe2\x8e\xbdn'
b'Username: vortex1 Password: Gq#qu3bF3'


# Vortex Level 1 → Level 2


In [24]:
0xFFFFFFFF - 0xCAFFFFFF

889192448

In [21]:
hex(-0xCA000000 & 0xFF000000)

'0x36000000'